In [ ]:
import subprocess
%run "{os.path.dirname(os.getcwd())}\general_functions\generalFunctions.ipynb" 

## Slide 1 Replacement Function

In [ ]:
def SkuShareByBrand(prs, assortmentTotalSorted, numOfDuplicates, column, position=0):
    """
    Updates PowerPoint slides with SKU and Value Share data by brand for a given market.
    
    Args:
    prs (Presentation): The PowerPoint presentation object.
    assortmentTotalSorted (dict): Dictionary containing sorted assortment data by market.
    numOfDuplicates (int): Number of slides to duplicate and update.
    position (int, optional): Starting position for slide updates. Default is 0.
    
    """
    for slidenum in range(numOfDuplicates):
        market = list(assortmentTotalSorted.keys())[slidenum]
        df = assortmentTotalSorted[market].copy()
        
        # Sort the dataframe by 'Value Share' in descending order
        df = df.sort_values('Value Share', ascending=False)
        
        # Ensure that 'Others' is the last row in the dataframe
        df = pd.concat([df[df[column] != 'Others'], df[df[column] == 'Others']]).reset_index(drop=True)
        
        # Get the shapes in the current slide
        shapes = prs.slides[slidenum + position].shapes
        # Update text in specific shapes
        headerNumber = get_shape_number(shapes, "SKU Share By Brand (Replace With SO WHAT)")
        titleNumber = get_shape_number(shapes, "SKU Share vs. Value Share | Category | National | P12M")
        shapes[titleNumber-1].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace('National', market.split(' | ')[1]).replace('Category', market.split(' | ')[0])
        
        # Format the text in the shapes
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
        shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
        
        # Create tables and charts from the shapes
        tables, charts = createTableAndChart(shapes)
        chart = charts[0].chart
        
        # Prepare chart data
        chart_data = CategoryChartData()
        chart_data.categories = df[column].tolist()
        chart_data.add_series('Value Share', df['Value Share'])
        chart_data.add_series('SKU Share', df['SKU Share'])
        
        # Calculate the index (SKU Share / Value Share) and handle division by zero
        chart_data.add_series('Index', ((df['SKU Share'] / df['Value Share']) * 100).replace(0, 1))
        
        # Replace the chart data with the prepared data
        chart.replace_data(chart_data)


## Slide 2 Replacement Function

In [ ]:
def CumulativeProductShare(prodORitem,prs, assortmentModifiedBrand, numOfDuplicates, position=0):
    """
    Updates PowerPoint slides with Cumulative Product Share data for a given market.
    
    Args:
    prs (Presentation): The PowerPoint presentation object.
    assortmentModifiedBrand (dict): Dictionary containing modified assortment data by brand.
    numOfDuplicates (int): Number of slides to duplicate and update.
    position (int, optional): Starting position for slide updates. Default is 0.
    
    """
    for slidenum in range(numOfDuplicates):
        # Get the market key and corresponding dataframe

        market = list(assortmentModifiedBrand.keys())[slidenum]
        df = assortmentModifiedBrand[market].copy()
        
        # Filter and sort the dataframe by 'Cumulative Product Share'
        df = df[df['Cumulative '+prodORitem+' Share'].notna()].sort_values(by='Cumulative '+prodORitem+' Share').reset_index(drop=True)
        
        # Get the shapes in the current slide
        shapes = prs.slides[slidenum + position].shapes
        
        # Update the title and data source in the slide
        headerNumber= get_shape_number(shapes, "Cumulative Product Share (Replace With SO WHAT)")
        titleNumber = get_shape_number(shapes, "Cumulative Product Share | Category | National | P12M")
        shapes[titleNumber - 1].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace('National', market.split(' | ')[1]).replace('Category', market.split(' | ')[0])
        
        # Format the text in the shapes
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
        shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
        
        # Create tables and charts from the shapes
        tables, charts = createTableAndChart(shapes)
        chart = charts[0].chart
        
        # Prepare chart data
        chart_data = CategoryChartData()
        chart_data.categories = [cat[:11] for cat in df['Top Brands']]
        chart_data.add_series('WD', df['WD'])
        chart_data.add_series('Cumulative '+prodORitem+' Share', df['Cumulative '+prodORitem+' Share'])
        
        # Replace the chart data with the prepared data
        chart.replace_data(chart_data)
        
        # Calculate the number of SKUs to reach 20%, 50%, and 80% cumulative product share
        count20 = 0
        count50 = 0
        count80 = 0
        startValue20 = df[df['Cumulative '+prodORitem+' Share'] >= .2]['Cumulative '+prodORitem+' Share'].min()
        count20 = df[df['Cumulative '+prodORitem+' Share'] <= startValue20].shape[0]
        startValue50 = df[df['Cumulative '+prodORitem+' Share'] >= .5]['Cumulative '+prodORitem+' Share'].min()
        count50 = df[df['Cumulative '+prodORitem+' Share'] <= startValue50].shape[0]
        startValue80 = df[df['Cumulative '+prodORitem+' Share'] >= .8]['Cumulative '+prodORitem+' Share'].min()
        count80 = df[df['Cumulative '+prodORitem+' Share'] <= startValue80].shape[0]
        
        # Update the color of the chart points based on the cumulative product share thresholds
        for i, point in enumerate(chart.series[1].points):
            point.format.fill.solid()
            if df['Cumulative '+prodORitem+' Share'][i] <= startValue80:
                color = RGBColor(0, 160, 151)
            elif df['Cumulative '+prodORitem+' Share'][i] < .95:
                color = RGBColor(126, 202, 196)
            else:
                color = RGBColor(174, 171, 171)
            point.format.fill.fore_color.rgb = color
        
        # Update the slide text with the calculated SKU counts
        skuNumber = get_shape_number(shapes, "#Skus to reach 80% is 117")
        shapes[skuNumber].text = shapes[skuNumber].text.replace('117', str(count80))
        shapes[skuNumber].text_frame.paragraphs[0].font.size = Pt(6.8)
        shapes[skuNumber].text_frame.paragraphs[0].font.name = 'Nexa Book'
        shapes[skuNumber].text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
        
        shapes[skuNumber + 1].text = shapes[skuNumber + 1].text.replace('59', str(count50))
        shapes[skuNumber + 1].text_frame.paragraphs[0].font.size = Pt(6.8)
        shapes[skuNumber + 1].text_frame.paragraphs[0].font.name = 'Nexa Book'
        shapes[skuNumber + 1].text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
        
        shapes[skuNumber + 2].text = shapes[skuNumber + 2].text.replace('22', str(count20))
        shapes[skuNumber + 2].text_frame.paragraphs[0].font.size = Pt(6.8)
        shapes[skuNumber + 2].text_frame.paragraphs[0].font.name = 'Nexa Book'
        shapes[skuNumber + 2].text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)


## Slide 3 Replacement Function

In [ ]:
# def cumulativeTop50(prodORitem,prs, cumulativeShareTop50, numOfDuplicates, position):
#     """
#     Updates PowerPoint slides with data for the top 50% cumulative share of products.

#     Args:
#     prs (Presentation): The PowerPoint presentation object.
#     cumulativeShareTop50 (dict): Dictionary containing cumulative share data filtered to top 50%.
#     numOfDuplicates (int): Number of slides to duplicate and update.
#     position (int): Starting position for slide updates.

#     """
#     for slide_num, key in enumerate(cumulativeShareTop50.keys()): 
#         # Get the dataframe for the current key and filter it
#         df = cumulativeShareTop50[key]
#         df = df[df['Cumulative '+prodORitem+' Share'].notnull()]
#         df50=df[df['Cumulative '+prodORitem+' Share'] <= 0.5]
#         dt=df[df['Cumulative '+prodORitem+' Share'] > 0.5]
#         minval_row = dt.loc[dt['Cumulative ' + prodORitem + ' Share'].idxmin()]

#         df = pd.concat([df50, minval_row.to_frame().T], ignore_index=True)        

#         df = df.sort_values(by='Cumulative '+prodORitem+' Share', ascending=True).reset_index(drop=True)
        
#         # Get the current slide to update
#         slide = prs.slides[slide_num + position]
        
#         # Split and strip the key to get market and category information
#         result = key.split('|')
#         result = [item.strip() for item in result]

#         # Get shapes from the slide
#         shapes = prs.slides[slide_num + position].shapes
        
#         # Update the heading and data source in the slide
#         headerNumber= get_shape_number(shapes, "Top 50% cumulative share (Replace With SO WHAT)")
#         titleNumber = get_shape_number(shapes, "Top 50% cumulative share | Category | National | P12M")
#         shapes[titleNumber - 1].text = data_source
#         shapes[titleNumber].text = shapes[titleNumber].text.replace('National', result[1]).replace('Category', result[0])
        
#         # Format the text in the heading shapes
#         shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
#         shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
#         shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)
#         shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'

#         # Create tables and charts from the slide shapes
#         tables, charts = createTableAndChart(slide.shapes)
#         chart = charts[0].chart
        
#         # Prepare chart data
#         chart_data = CategoryChartData()
#         chart_data.categories = df['Top Brands']
#         chart_data.add_series('WD', df['WD'])
#         chart_data.add_series('Cumulative '+prodORitem+' Share', df['Cumulative '+prodORitem+' Share'])
        
#         slideType = ''
#         if slideType == 'Cumulative':
#             chart.value_axis.visible = False
#             chart.value_axis.has_title = False
#             chart_data.add_series(prodORitem+' Sales Rate', [None])
#         else:
#             chart_data.add_series(prodORitem+' Sales Rate', df[prodORitem+' Sales Rate'])
        
#         # Replace the chart data with the prepared data
#         chart.replace_data(chart_data)
        
#         # Update the color of chart points based on whether the brand is a client brand
#         for idx, point in enumerate(chart.series[1].points):
#             point.format.fill.solid()
#             point.format.fill.fore_color.rgb = RGBColor(0, 160, 151)
#             if df['Top Brands'].iloc[idx] in client_brands:
#                 point.format.fill.fore_color.rgb = RGBColor(255, 191, 191)


## Slide 4 Replacement Function

In [ ]:
def brandCumulativeProductShare(prodORitem,prs, assortmentClient, numOfDuplicates, position=0):
    """
    Updates PowerPoint slides with cumulative product share data for specified brands.

    Args:
    prs (Presentation): The PowerPoint presentation object.
    assortmentClient (dict): Dictionary containing cumulative share data for specific brands.
    numOfDuplicates (int): Number of slides to duplicate and update.
    position (int): Starting position for slide updates.

    """
    for slide_num, key in enumerate(assortmentClient.keys()):
        # Get the dataframe for the current key
        df = assortmentClient[key]        
        # Select the current slide to update
        slide = prs.slides[slide_num + position]
        
        # Create tables and charts for the slide
        tables, charts = createTableAndChart(slide.shapes)
        
        # Split and strip the key to get market, category, and brand information
        result = key.split('|')
        result = [item.strip() for item in result]
        
        # Get shapes from the slide
        shapes = prs.slides[slide_num + position].shapes
        
        # Update the heading and data source in the slide
        headerNumber= get_shape_number(shapes, "Brand Cumulative Product Share (Replace With SO WHAT)")
        titlNumber = get_shape_number(shapes, "Cumulative Product Share | Category | Brand | National | P12M")
        shapes[titlNumber - 1].text = data_source
        shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
        
        # Update the text in the title shape
        shapes[titlNumber].text = shapes[titlNumber].text.replace('Category', key.split(' | ')[0]).replace('Brand', key.split(' | ')[2]).replace('National', key.split(' | ')[1])
        shapes[titlNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titlNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
    
        # Filter and sort the dataframe for the chart
        new = df[df['Cumulative Share'] > 0]
        new.sort_values(by='Cumulative Share', ascending=True, inplace=True)
        new = new.reset_index(drop=True)
        
        # Get the chart from the slide
        chart = charts[0].chart
        
        # Prepare chart data
        chart_data = CategoryChartData()
        chart_data.categories = new[prodORitem]
        chart_data.add_series('WD', new['WD'])
        chart_data.add_series('Cumulative Share', new['Cumulative Share'])
        chart_data.add_series(prodORitem+' Sales Rate', new[prodORitem+' Sales Rate'])
        
        # Replace the chart data with the prepared data
        chart.replace_data(chart_data)
        
        # Update the color of chart points based on cumulative product share
        for i, point in enumerate(chart.series[1].points):
            point.format.fill.solid()
            if new['Cumulative Share'][i] <= 0.8:
                color = RGBColor(0, 160, 151)
            elif new['Cumulative Share'][i] < 0.95:
                color = RGBColor(126, 202, 196)
            else:
                color = RGBColor(174, 171, 171)
            point.format.fill.fore_color.rgb = color
        
        # Replace the chart data with updated colors
        chart.replace_data(chart_data)


## Slide 5 Replacement Function

In [ ]:
def top20CumulativeShare(prodORitem, prs, assortmentModifiedBrand, numOfDuplicates, client_brands, position=0):
    """
    Updates PowerPoint slides with the top 20 cumulative product share data.

    Args:
    prs (Presentation): The PowerPoint presentation object.
    assortmentModifiedBrand (dict): Dictionary containing modified assortment data by brand.
    numOfDuplicates (int): Number of slides to duplicate and update.
    client_brands (list): List of client brand names to highlight in green.
    position (int): Starting position for slide updates.
    """
    from pptx.dml.color import RGBColor
    from pptx.util import Pt
    from pptx.enum.text import PP_ALIGN

    for slidenum in range(numOfDuplicates):
        market = list(assortmentModifiedBrand.keys())[slidenum]
        df = assortmentModifiedBrand[market]

        # Filter and sort the DataFrame for the top 20 cumulative product shares
        df = df[df[f'Cumulative {prodORitem} Share'].notna()].sort_values(
            by=f'Cumulative {prodORitem} Share', ascending=True).head(20)

        shapes = prs.slides[slidenum + position].shapes

        # Update slide heading
        headerNumber = get_shape_number(shapes, "Top 20 cumulative share (Replace With SO WHAT)")
        titlNumber = get_shape_number(shapes, "Top 20 cumulative share products | Category | National | P12M")

        shapes[titlNumber].text = shapes[titlNumber].text.replace(
            'National', market.split(' | ')[1]).replace('Category', market.split(' | ')[0])
        shapes[titlNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titlNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
        shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'

        # Update data source text
        shapes[titlNumber - 1].text = data_source

        # Create tables and charts
        tables, charts = createTableAndChart(shapes)
        table = tables[0].table

        # Remove extra rows
        num_rows_to_remove = len(table.rows) - df.shape[0] - 1
        table_height = get_table_height(table)

        for _ in range(num_rows_to_remove):
            if len(table.rows) > 1:
                row = table.rows[1]
                remove_row(table, row)

        # Adjust row height
        if num_rows_to_remove:
            total_row_height = table_height - table.rows[0].height
            num_rows = len(table.rows) - 1
            if num_rows > 0:
                cell_height = total_row_height / num_rows
                for row in range(1, len(table.rows)):
                    table.rows[row].height = int(cell_height)

        # Populate table with data
        for i, row in enumerate(table.rows):
            if i == 0:
                # Header row
                for j, cell in enumerate(row.cells):
                    if j == 4:
                        cell.text = cell.text.replace("Currency", currency).replace("Unit", unit)
                        para = cell.text_frame.paragraphs[0]
                        para.font.name = 'Nexa Bold'
                        para.font.size = Pt(8)
                        para.alignment = PP_ALIGN.CENTER
                continue

            # Data row
            row_idx = i - 1
            brand_name = df['Top Brands'].iloc[row_idx]
            is_client_brand = brand_name in client_brands

            for j, cell in enumerate(row.cells):
                if j == 0:
                    cell.text = brand_name
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                elif j == 1:
                    cell.text = str(df[prodORitem].iloc[row_idx])
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT
                elif j == 2:
                    cell.text = f"{int(round(df['WD'].iloc[row_idx] * 100))}%"
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                elif j == 3:
                    share_val = df[f'Cumulative {prodORitem} Share'].iloc[row_idx]
                    cell.text = f"{round(share_val * 100, 1)}%"
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                elif j == 4:
                    rate_val = df[f'{prodORitem} Sales Rate'].iloc[row_idx]
                    cell.text = f"{round(rate_val / unitDeviation, 1)}"
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

                # Style the font
                para = cell.text_frame.paragraphs[0]
                para.font.name = 'Nexa Book'
                para.font.size = Pt(8)

                # Apply green color if client brand
                if is_client_brand:
                    fill = cell.fill
                    fill.solid()
                    fill.fore_color.rgb = RGBColor(203, 234, 231)

## Slide 6 Replacement Function

In [19]:
def SKUProductivityAnalysis(prodORitem,prs, assortmentClientBrand, numOfDuplicates, position=0):
    """
    Updates PowerPoint slides with SKU productivity analysis.

    Args:
    prs (Presentation): The PowerPoint presentation object.
    assortmentClientBrand (dict): Dictionary containing assortment data by brand.
    numOfDuplicates (int): Number of slides to duplicate and update.
    position (int): Starting position for slide updates.

    """
    for slidenum in range(numOfDuplicates):
        # Get the market key and corresponding DataFrame
        market = list(assortmentClientBrand.keys())[slidenum]
        df = assortmentClientBrand[market].reset_index(drop=True)

        # Select the current slide to update
        shapes = prs.slides[slidenum + position].shapes
        
        # Find the shape numbers for title and header
        
        titlNumber = get_shape_number(shapes, 'Product Sales Rate, Trade Margin % and Net Sales | Category | Brand | National | P12M\nBubble Size: Net Sales')
        headerNumber = get_shape_number(shapes, "SKU Productivity Analysis with TM% (Replace with So What)")
        
        # Update the data source text
        shapes[titlNumber - 1].text = data_source
        
        # Update the header with specific font size and name
        shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
        
        # Update the title text with market, category, and brand information
        shapes[titlNumber].text = shapes[titlNumber].text.replace('Category', market.split(' | ')[0]).replace('Brand', market.split(' | ')[2]).replace('National', market.split(' | ')[1])

        # Update the font style and size for paragraphs in the title text frame
        for i, par in enumerate(shapes[titlNumber].text_frame.paragraphs):
            shapes[titlNumber].text_frame.paragraphs[i].font.name = 'Nexa (Headings)'
            shapes[titlNumber].text_frame.paragraphs[i].font.size = Pt(11)  # Keep the same font size
        
        # Create tables and charts for the slide
        tables, charts = createTableAndChart(shapes)
        chart = charts[0].chart
        
        # Populate the chart data
        chart_data = BubbleChartData()
        series_1 = chart_data.add_series('sku productivity index')
        for i in range(df.shape[0]):
            series_1.add_data_point(df['Trade Margin %'][i], df[prodORitem+' Sales Rate'][i], df['Net Sales'][i])

        # Replace the chart data
        chart.replace_data(chart_data)
        xlsx_file=BytesIO()
        with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
                chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
                worksheet.write(0, 4, "labels")
                worksheet.write_column(1, 4, df[prodORitem], None)

        chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())


        # # Add data labels to the chart points
        # for j, point in enumerate(chart.series[0].points):
        #     point.data_label.text_frame.text = df[prodORitem][j]


## Slide 7 Replacement Function

In [20]:
def SKUWithWD(prodORitem,prs, assortmentClientBrand, numOfDuplicates, position=0):
    """
    Updates PowerPoint slides with SKU analysis including WD (weighted distribution).

    Args:
    prs (Presentation): The PowerPoint presentation object.
    assortmentClientBrand (dict): Dictionary containing assortment data by brand.
    numOfDuplicates (int): Number of slides to duplicate and update.
    position (int): Starting position for slide updates.

    """
    for slidenum in range(numOfDuplicates):
        # Get the market key and corresponding DataFrame
        market = list(assortmentClientBrand.keys())[slidenum]
        df = assortmentClientBrand[market].reset_index(drop=True)

        # Select the current slide to update
        shapes = prs.slides[slidenum + position].shapes
        
        # Find the shape numbers for title and header
        titlNumber = get_shape_number(shapes, "Product Sales Rate, WD and Net Sales | Category | Brand | National | P12M\nBubble Size: Net Sales")
        headerNumber = get_shape_number(shapes, "SKU Productivity Analysis with WD (Replace with So What)")

        # Update the data source text
        shapes[titlNumber - 1].text = data_source
        
        # Update the font size and name for the paragraphs in the title text frame
        shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
        
        # Update the title text with market, category, and brand information
        shapes[titlNumber].text = shapes[titlNumber].text.replace('Category', market.split(' | ')[0]).replace('Brand', market.split(' | ')[2]).replace('National', market.split(' | ')[1])

        # Update the font style and size for paragraphs in the title text frame
        for i, par in enumerate([0, 1]):
            shapes[titlNumber].text_frame.paragraphs[i].font.name = 'Nexa (Headings)'
            shapes[titlNumber].text_frame.paragraphs[i].font.size = Pt(11)  # Keep the same font size
      
        # Create tables and charts for the slide
        tables, charts = createTableAndChart(shapes)
        
        chart = charts[0].chart
        chart_data = BubbleChartData()

        # Populate the chart data
        series_1 = chart_data.add_series("Value RoS")
        for i in range(df.shape[0]):
            series_1.add_data_point(df['WD'][i], df[prodORitem+' Sales Rate'][i], df['Net Sales'][i])

        # Replace the chart data
        chart.replace_data(chart_data)
        xlsx_file=BytesIO()
        with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
            chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
            worksheet.write(0, 4, "labels")
            worksheet.write_column(1, 4, df[prodORitem], None)

        chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

        # # Add data labels to the chart points
        # for j, point in enumerate(chart.series[0].points):
        #     point.data_label.text_frame.text = df[prodORitem][j]


### Slide 8 Fair Share (New Slide)

In [ ]:
def fair_share(brand_total,prs ,scope, position=0,category=''):
    slidenum = 0
    for key, df in brand_total.items():

        market = key.split(' | ')[0]
        brand = key.split(' | ')[1]

        shapes = prs.slides[slidenum + position].shapes

        titleNumber = get_shape_number(shapes, "Sectors fair share | Sb/Sc | Prince | National | P12M")
        headerNumber = get_shape_number(shapes, "Sectors Fair Share (Replace with So What)")
        shapes[titleNumber-1].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace('Sectors', scope)
        shapes[titleNumber].text = shapes[titleNumber].text.replace('Sb/Sc', category)
        shapes[titleNumber].text = shapes[titleNumber].text.replace('Prince',brand)
        shapes[titleNumber].text = shapes[titleNumber].text.replace('National',market)
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
        
        shapes[headerNumber].text_frame.paragraphs[0].runs[0].text=shapes[headerNumber].text_frame.paragraphs[0].runs[0].text.replace('Sectors',scope)
        shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'

        tables, charts = createTableAndChart(shapes)
        table = tables[0].table

        # Calculate the number of rows to remove from the table

        num_rows_to_remove = len(table.rows) - df.shape[0] -1
        table_height = get_table_height(table)


        for i in range (len(table.rows)-1, df.shape[0], -1):
                remove_row(table,table.rows[i])
                        

        if num_rows_to_remove:
                total_row_height = table_height - table.rows[0].height
                num_rows = len(table.rows) - 1  # Exclude the first row
                if num_rows > 0:
                        cell_height = total_row_height / num_rows
                        for row in range(1, len(table.rows)):
                                table.rows[row].height = int(cell_height)

        for i, row in enumerate(table.rows):
                for j, cell in enumerate(row.cells):
                        if i == 0 and j >1:
                                cell.text = cell.text.replace("Prince", brand)
                                for l in range(2):
                                        cell.text_frame.paragraphs[l].alignment = PP_ALIGN.CENTER
                                        cell.text_frame.paragraphs[l].font.name = 'Nexa (Headings)'
                                        cell.text_frame.paragraphs[l].font.size = Pt(9)
                                        cell.text_frame.paragraphs[l].font.bold = True

                        if i > 0 and j == 0:
                                cell.text = df[scope][i-1]
                                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                cell.text_frame.paragraphs[0].font.name = 'Nexa (Headings)'
                                cell.text_frame.paragraphs[0].font.size = Pt(8)
                                cell.text_frame.paragraphs[0].font.bold = True

                        if i > 0 and j == 1:
                                value =  int(format_number(df["Av No SKUs_total"][i-1], use_decimals=True, decimals=0))
                                cell.text = str(value) if value >=0 else '0'
                                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                cell.text_frame.paragraphs[0].font.size = Pt(8)
                                
                        if i > 0 and j == 2:
                                value = int(format_number(df["Av No SKUs_brand"][i-1], use_decimals=True, decimals=0))
                                cell.text = str(value) if value >=0 else '0'
                                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                cell.text_frame.paragraphs[0].font.size = Pt(8)

                        if i > 0 and j == 3:
                                value = int(format_number(df["SKU Share"][i-1]*100, use_decimals=True, decimals=0))
                                cell.text = str(value)+'%' if value >= 0 else '0'
                                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                cell.text_frame.paragraphs[0].font.size = Pt(8)

                        if i > 0 and j == 4: 
                                value = int(format_number(df["Value Share"][i-1]*100, use_decimals=True, decimals=0))
                                cell.text = str(value)+'%' if value >= 0 else '0'
                                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                cell.text_frame.paragraphs[0].font.size = Pt(8)
                        
                        if i > 0 and j ==5:
                                value = int(format_number(df['Fair Share'][i-1], use_decimals=True, decimals=0)) if df['Value Share'][i-1] >= 0 and df['Av No SKUs_total'][i-1] >=0 else 0
                                cell.text = str(value) if value >= 0.0 else '0'
                                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                cell.text_frame.paragraphs[0].font.size = Pt(8)
                        
                        if i > 0 and j == 6:
                                value = int(format_number(df["Listing Opportunity"][i-1], use_decimals=True, decimals=0)) if df['Value Share'][i-1] >= 0  else 0
                                value = int(value) if value >= 0 else 0 

                                value = "-" if value == 0 else str(value)

                                cell.text =value
                                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                                cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                                cell.text_frame.paragraphs[0].font.size = Pt(8)
        slidenum +=1                        
                        
                        
                        
                        